# АНСАМБЛИ МОДЕЛЕЙ

In [58]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from jupyterthemes import jtplot
from sklearn.model_selection import KFold, StratifiedKFold, cross_validate, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import auc, roc_curve, roc_auc_score, r2_score, mean_squared_error, mean_absolute_error


from sklearn.metrics import auc, roc_curve, roc_auc_score
%matplotlib inline
jtplot.style()

Домашняя работа
Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
Построить случайный лес, вывести важность признаков
Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
В качестве решения: Jupyter notebook с кодом, комментариями и графиками

In [59]:
data = pd.read_csv('train_home.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


SalePrice-цена продажи объекта недвижимости в долларах США. Это целевая переменная, которую вы пытаетесь предсказать.
MSSubClass: класс здания
MSZoning: общая классификация зонирования
LotFrontage: линейные ноги улицы, соединенной с недвижимостью
LotArea: Размер лота в квадратных футах
Улица: тип подъездной дороги
Аллея: тип доступа в аллею
LotShape: общая форма собственности
LandContour: плоскостность объекта недвижимости
Коммунальные услуги: тип доступных коммунальных услуг
LotConfig: конфигурация лота
Оползень: уклон объекта недвижимости
Район: физические места в пределах города Эймс
Условие1: близость к главной дороге или железной дороге
Условие2: близость к главной дороге или железной дороге (если есть вторая)
BldgType: тип жилья
HouseStyle: стиль жилища
Общее качество: общий материал и качество отделки
OverallCond: общая оценка состояния
YearBuilt: первоначальная дата строительства
YearRemodAdd: дата реконструкции
RoofStyle: Тип крыши
RoofMatl: материал крыши
Exterior1st: наружное покрытие дома
Exterior2nd: наружное покрытие дома (если более одного материала)
MasVnrType: тип облицовки каменной кладки
MasVnrArea: площадь облицовки каменной кладки в квадратных футах
ExterQual: качество внешнего материала
ExterCond: настоящее состояние материала на внешней стороне
Фундамент: тип фундамента
BsmtQual: высота подвала
BsmtCond: общее состояние подвала
BsmtExposure: стены подвала уровня дорожки или сада
BsmtFinType1: качество отделанной площади подвала
BsmtFinSF1: Тип 1 законченные квадратные футы
BsmtFinType2: качество второй готовой области (если есть)
BsmtFinSF2: Тип 2 законченные квадратные футы
BsmtUnfSF: недостроенные квадратные метры подвальной площади
TotalBsmtSF: общая площадь подвала в квадратных футах
Отопление: тип отопления
HeatingQC: качество и состояние отопления
CentralAir: центральное кондиционирование воздуха
Электрическое: электрическая система
1stFlrSF: первый этаж квадратных футов
2ndFlrSF: второй этаж квадратных футов
LowQualFinSF: низкое качество готовых квадратных футов (все этажи)
GrLivArea: выше класса (земля) жилая площадь квадратных футов
BsmtFullBath: подвал с полными ванными комнатами
BsmtHalfBath: полуподвальные ванные комнаты
FullBath: полные ванные комнаты выше класса
HalfBath: половинные ванны выше класса
Спальня: Количество спален над цокольным этажом
Кухня: количество кухонь
KitchenQual: качество кухни
TotRmsAbvGrd: всего номеров выше класса (не включает ванные комнаты)
Функционал: рейтинг домашней функциональности
Камины: количество каминов
FireplaceQu: качественный камин
Тип гаража: расположение гаража
GarageYrBlt: Год постройки гаража
GarageFinish: внутренняя отделка гаража
Гаражные вагоны: размер гаража в вместимости автомобиля
GarageArea: размер гаража в квадратных футах
GarageQual: качество гаража
GarageCond: состояние гаража
PavedDrive: мощеная подъездная дорожка
WoodDeckSF: площадь деревянной палубы в квадратных футах
OpenPorchSF: открытая площадь крыльца в квадратных футах
EnclosedPorch: закрытая площадь крыльца в квадратных футах
3SsnPorch: Трехсезонная площадь крыльца в квадратных футах
ScreenPorch: площадь крыльца экрана в квадратных футах
Площадь бассейна: Площадь бассейна в квадратных футах
PoolQC: качество бассейна
Забор: качество забора
MiscFeature: разные функции, не охватываемые другими категориями
MiscVal: $значение разных функций
Мосолд: Продано За Месяц
YrSold: Проданный Год
SaleType: тип продажи
SaleCondition: условие продажи

In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [61]:
data.describe().transpose()[1:]

,count,mean,std,min,25%,50%,75%,max
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotFrontage,1201.0,70.049958,24.284752,21.0,59.00,69.0,80.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
MasVnrArea,1452.0,103.685262,181.066207,0.0,0.00,0.0,166.00,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0
BsmtFinSF2,1460.0,46.549315,161.319273,0.0,0.00,0.0,0.00,1474.0


In [62]:
data.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [63]:
cat_feat = list(data.dtypes[data.dtypes == object].index)
data[cat_feat] = data[cat_feat].fillna('nan')
#num_feat = [f for f in data if f not in (cat_feat + ['ID', 'SalePrice'])]

cat_feat = data[cat_feat].apply(LabelEncoder().fit_transform)
num_feat = [f for f in data if f not in cat_feat]
data[num_feat] = data[num_feat].fillna(999)
num_feat = data[num_feat]

In [64]:
data = pd.concat([num_feat, cat_feat], axis=1)
cat_feat = cat_feat.columns
num_feat = num_feat.columns

y = data['SalePrice']
X = data.drop(['SalePrice', 'Id'], axis=1)
data_X = data.drop(['SalePrice'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y)

#обучим лес деревьев
model_rf = RandomForestRegressor(n_estimators=500, max_features=15)
model_rf.fit(X_train, y_train)
y_pred_train_rf = model_rf.predict(X_train)
y_pred_test_rf = model_rf.predict(X_test)
mean_squared_error(y_train, y_pred_train_rf), mean_squared_error(y_test, y_pred_test_rf)
r2_score(y_train, y_pred_train_rf), r2_score(y_test, y_pred_test_rf)                                                                   
                                                                       

(0.9814915161330311, 0.8561491210195373)

In [65]:
imp = pd.Series(model_rf.feature_importances_)
imp.sort_values(ascending=False).head(4)

print(f'корреляция очень мала. \nна первом месте {data_X.take([3], axis=1)[:1]}\nна втором {data_X.take([15], axis=1)[:1]}\n')                                                     
                                                                       

корреляция очень мала. 
на первом месте    LotArea
0     8450
на втором    LowQualFinSF
0             0



In [66]:
splitter = KFold(n_splits=10, random_state=555)
#первым в выборку идет 
for train_index, test_index in splitter.split(data, y):
    X_train = data.iloc[train_index]
    X_test = data.iloc[test_index]
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

/Users/natalaurina/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [67]:
scaler = StandardScaler()
scaler.fit(X_train[num_feat])

X_train[num_feat] = scaler.transform(X_train[num_feat])
X_test[num_feat] = scaler.transform(X_test[num_feat])

/Users/natalaurina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/natalaurina/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/natalaurina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

## Стекинг

Используем идею K-Fold проверки. Выборку разбиваем на фолды, затем последовательно перебираем фолды и обучаем базовые алгоритмы на всех фолдах, кроме одного, а на оставшемся получаем ответы базовых алгоритмов и трактуем их как значения соответствующих признаков на этом фолде. Для получения метапризнаков объектов тестовой выборки базовые алгоритмы обучают на всей обучающей выборке и берут их ответы на тестовой.

<img src="img/stacking-2b.png" style="max-width: 680px; display: inline" />

### Смешаем дерево и логистическую регрессию


### Готовим признаки

## обучим стекинг

In [68]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier

In [69]:
classifier = StackingRegressor(
    [
        ('lr', LinearRegression()),
        ('rf', RandomForestRegressor()),
        ('rg',Ridge())
    ],
RandomForestRegressor())

In [70]:
classifier.fit(X_train, y_train)

StackingRegressor(estimators=[('lr', LinearRegression()),
                              ('rf', RandomForestRegressor()),
                              ('rg', Ridge())],
                  final_estimator=RandomForestRegressor())

In [71]:
classifier.named_estimators_['rf']

RandomForestRegressor()

In [72]:
y_pred_rf = classifier.named_estimators_['rf'].predict(X_test)

In [73]:
y_pred_lr = classifier.named_estimators_['lr'].predict(X_test)

In [74]:
y_pred_rg = classifier.named_estimators_['rg'].predict(X_test)

In [75]:
y_pred = classifier.predict(X_test)

In [83]:
preds_train, preds_test = classifier.predict(X_train), classifier.predict(X_test)
r2_score(y_train, preds_train), r2_score(y_test, preds_test)

(0.9997924073741797, 0.9999757180019531)

In [88]:
r2_score(y_test, y_pred_rg), r2_score(y_test, y_pred_lr), r2_score(y_test, y_pred_rf)

(0.9999943732293003, 1.0, 0.999905934166116)